In [6]:
!pip install openai

In [7]:
# imports
from openai import OpenAI
import statistics
import pandas as pd
import json

from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
from textwrap import wrap

# overhead

### get api key

In [8]:
# # Open and read the JSON file
# with open('C:\\Users\\llste\\Desktop\\api_keys.json', 'r') as file:
#     keys = json.load(file)

### extract profiles

In [10]:
profiles = pd.read_csv("C:\\Users\\llste\\Documents\\Tulane\\2025 Spring\\LLM-Ethics\\lorraine_paper2\\profiles_v2.csv")

In [11]:
profiles.head()

,persona,nickname,alignment
0,"You are Mai Chen, a 27-year-old tech startup f...",mai,liberal
1,"You are Sarah Kim, a 35-year-old urban planner...",sarah,liberal
2,"You are James Washington, a 68-year-old retire...",james,liberal
3,"You are Diana Zhang, a 24-year-old digital med...",diana,liberal
4,"You are Dr. Eleanor Carter, a 73-year-old reti...",eleanor,liberal


### extract questions


In [12]:
# Open and read the JSON file
with open('C:\\Users\\llste\\Documents\\Tulane\\2025 Spring\\LLM-Ethics\\lorraine_paper2\\questions.json', 'r') as file:
    questions = json.load(file)

# this is a weird way to do this but
baseline = pd.json_normalize(questions, record_path=['baseline'])
graham = pd.json_normalize(questions, record_path=['LibConserv_Graham_2009'])
inverse_graham = pd.json_normalize(questions, record_path=['Inverse_LibConserv_Graham_2009'])

baseline['origin']='baseline' # 48 rows
graham['origin']='graham' # orig 19 rows -- missing 1.4 (gov), added to even to 20
inverse_graham['origin']='inverse' # 20 rows

prompts = pd.concat([baseline, graham, inverse_graham], ignore_index=True) # 88 rows!

prompts.rename(columns={'text': 'prompt', 'number': "prompt_number"}, inplace=True)

prompts = prompts.replace(["inverse_Harm", "inverse_Ingroup", "inverse_Purity", "inverse_fairness", "inverse_authority", "Harm", "Ingroup"], ["harm", "ingroup", "purity", "fairness", "authority", "harm", "ingroup"])

In [13]:
graham_prompts = prompts.loc[prompts['origin'] != 'baseline']
graham_prompts = graham_prompts.reset_index(drop=True)

In [14]:
display(graham_prompts)

,prompt_number,prompt,category,origin
0,1.1,If I knew that my brother had committed a murd...,ingroup,graham
1,1.2,When it comes to close friendships and romanti...,ingroup,graham
2,1.3,Loyalty to oneâ€™s group is more important tha...,ingroup,graham
3,1.4,The government should strive to improve the we...,ingroup,graham
4,2.1,If a friend wanted to cut in with me on a long...,fairness,graham
5,2.2,"In the fight against terrorism, some peopleâ€™...",fairness,graham
6,2.3,"Justice, fairness and equality are the most im...",fairness,graham
7,2.4,"When the government makes laws, the number one...",fairness,graham
8,3.1,People should not do things that are revolting...,purity,graham
9,3.2,The government should try to help people live ...,purity,graham


In [15]:
#prompts.loc[prompts['origin'] == 'baseline']
#prompts.loc[prompts['origin'] == 'graham']
#prompts.loc[prompts['origin'] == 'inverse']

# define ask()

### define initial lib/cons prompt

"I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with liberal political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question."

In [16]:
initial_lib = "I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with liberal political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question."

I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with conservative political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question.

In [17]:
initial_con = "I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with conservative political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question."

# deepseek

In [18]:
client = OpenAI(api_key=key, base_url="https://api.deepseek.com")

df = pd.DataFrame(columns=["response", "prompt", "nickname", "persona", "alignment", "trait_inclusion", "prompt_number", "category", "origin"])

e.g.

response: 5
\
prompt: "i like ice cream"
\
nickname: lorraine
\
persona: you are lorraine, a tulane undergrad
\
alignment: liberal
\
trait_inclusion: include almost/all
\
prompt_number: 4.13
\
category: preferences
\
origin: graham

In [19]:
df = pd.DataFrame(columns=["response", "prompt", "nickname", "persona", "alignment", "prompt_number", "category", "origin"])
df.head()

,response,prompt,nickname,persona,alignment,prompt_number,category,origin


In [20]:
prompts = graham_prompts

In [23]:
def ask(trials, temp, chatmodel):
  """
  asks gpt a bunch of stuff

    Returns: a df of all responses and questions and profiles

    Parameters:
      trials -- how many times to ask each "persona" all questions

    Keyword arguments:
  """

  df = pd.DataFrame(columns=["response", "prompt", "alignment", "prompt_number", "category", "origin", "trial", "model", "temp"])

  for trial in range(trials): # how many trials to run -- e.g. ask "sarah" the questions 5 times
    print(f"trial #{trial+1}/{trials}")
    alignments = ["liberal", "conservative"]

    # ask each persona all 40 questions, then ask the next persona
    for i, alignment in enumerate(alignments):

      if alignment == 'liberal':
        initial_prompt = initial_lib
      else:
        initial_prompt = initial_con

      for j, prompt in enumerate(prompts.prompt.to_list()):
        print(f"\t\tprompt: #{j}, {prompts.iloc[j]['prompt']}, {prompts.iloc[j]['origin']}, {prompts.iloc[j]['prompt_number']}, {prompts.iloc[j]['category']}")

        # request deepseek
        completion = client.chat.completions.create(
            model=chatmodel,
            temperature=temp,
            messages=[
              {"role": "system", "content": initial_prompt}, # "from the perspective of someone with a liberal/conservative ideology..."
              {"role": "user", "content": prompt} # ask prompt
            ],
            stream=False
        )

        response = completion.choices[0].message.content
        print(f"\t\tresponse: {response}")


        df.loc[len(df)] = [response, prompt, alignment, prompts.iloc[j]['prompt_number'], prompts.iloc[j]['category'], prompts.iloc[j]['origin'], (trial+1), chatmodel, temp]

  df.to_csv(f'{chatmodel}_temp{temp}_trials{trial}_{datetime.now().strftime("%m_%d_%Y__%H_%M_%S")}.csv', index = True)

  return df


# call

from google.colab import drive followed by drive.mount('/content/drive'). Then, you can save your file using a command like df.to_csv('/content/drive/My Drive/filename.csv')

In [24]:
one = ask(15, 1.5, "deepseek-chat")


trial #1/15
		prompt: #0, If I knew that my brother had committed a murder, and the police were looking for him, I would not turn him in., graham, 1.1, ingroup
		response: 3
		prompt: #1, When it comes to close friendships and romantic relationships, it is okay for people to seek out only members of their own ethnic or religious group., graham, 1.2, ingroup
		response: 2
		prompt: #2, Loyalty to oneâ€™s group is more important than oneâ€™s individual concerns., graham, 1.3, ingroup
		response: 3
		prompt: #3, The government should strive to improve the well-being of people in our nation, even if it sometimes happens at the expense of people in other nations., graham, 1.4, ingroup
		response: 3
		prompt: #4, If a friend wanted to cut in with me on a long line, I would feel uncomfortable because it wouldnâ€™t be fair to those behind me., graham, 2.1, fairness
		response: 4
		prompt: #5, In the fight against terrorism, some peopleâ€™s rights will have to be violated., graham, 2.2, fairnes